# Part I. ETL Pipeline for Pre-Processing the Files

In [19]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [20]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/genie/project1b


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [21]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [22]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [23]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [24]:
# TO-DO: Create a Keyspace 
try:
    session.execute (
        """CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = {
            'class' : 'SimpleStrategy', 'replication_factor' : 1
        }
        """
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [25]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace ('udacity')
except Exception as e:
    print(e)


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [26]:
query = "drop TABLE IF EXISTS music_by_sessionid_iteminsession "
session.execute(query)

In [27]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_by_sessionid_iteminsession "
query = query + """(
sessionId int, itemInsession int, artist text, song text, length float, 
PRIMARY KEY(sessionId, itemInSession))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)      

In [28]:
## TO-DO: Assign the INSERT statements into the `query` variable
# Insert data into table for query 1
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_by_sessionid_iteminsession (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        print(line[8])
        print((line[0], line[1], line[2], line[3], line[4],float(line[5]),line[6],line[7],line[8],line[9],line[10]))
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

583
('Harmonia', 'Ryan', 'M', '0', 'Smith', 655.77751, 'free', 'San Jose-Sunnyvale-Santa Clara, CA', '583', 'Sehr kosmisch', '26')
583
('The Prodigy', 'Ryan', 'M', '1', 'Smith', 260.07465, 'free', 'San Jose-Sunnyvale-Santa Clara, CA', '583', 'The Big Gundown', '26')
583
('Train', 'Ryan', 'M', '2', 'Smith', 205.45261, 'free', 'San Jose-Sunnyvale-Santa Clara, CA', '583', 'Marry Me', '26')
597
('Sony Wonder', 'Samuel', 'M', '0', 'Gonzalez', 218.06975, 'free', 'Houston-The Woodlands-Sugar Land, TX', '597', 'Blackbird', '61')
602
('Van Halen', 'Tegan', 'F', '2', 'Levine', 289.38404, 'paid', 'Portland-South Portland, ME', '602', 'Best Of Both Worlds (Remastered Album Version)', '80')
602
('Magic Sam', 'Tegan', 'F', '3', 'Levine', 132.04853, 'paid', 'Portland-South Portland, ME', '602', 'Call Me If You Need Me', '80')
602
('Edward Sharpe & The Magnetic Zeros', 'Tegan', 'F', '4', 'Levine', 306.31138, 'paid', 'Portland-South Portland, ME', '602', 'Home', '80')
602
('Usher featuring will.i.am', 

#### Do a SELECT to verify that the data have been inserted into each table

In [29]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
SELECT artist, song, length 
FROM music_by_sessionid_iteminsession
WHERE sessionId = 338
AND itemInSession = 4
"""
query = session.execute(query)
print(query)
for row in query:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [30]:
query = "drop TABLE IF EXISTS music_by_artist_song_user "
session.execute(query)
query = "CREATE TABLE IF NOT EXISTS music_by_artist_song_user "
query = query + """(
userId int, sessionId int, artist text, song text, firstName text, lastName text, 
PRIMARY KEY(userId, sessionId))
"""
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [31]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_by_artist_song_user (userId, sessionId, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        print(line[8])
        print((int(line[10]), int(line[8]), line[0], line[9], line[1],line[4]))
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4]))

query = """
SELECT artist, song, firstName, lastName 
FROM music_by_artist_song_user
WHERE userId = 10
AND sessionId = 182
"""
query = session.execute(query)
print(query)
for row in query:
    print(row)

                    

583
(26, 583, 'Harmonia', 'Sehr kosmisch', 'Ryan', 'Smith')
583
(26, 583, 'The Prodigy', 'The Big Gundown', 'Ryan', 'Smith')
583
(26, 583, 'Train', 'Marry Me', 'Ryan', 'Smith')
597
(61, 597, 'Sony Wonder', 'Blackbird', 'Samuel', 'Gonzalez')
602
(80, 602, 'Van Halen', 'Best Of Both Worlds (Remastered Album Version)', 'Tegan', 'Levine')
602
(80, 602, 'Magic Sam', 'Call Me If You Need Me', 'Tegan', 'Levine')
602
(80, 602, 'Edward Sharpe & The Magnetic Zeros', 'Home', 'Tegan', 'Levine')
602
(80, 602, 'Usher featuring will.i.am', 'OMG', 'Tegan', 'Levine')
602
(80, 602, 'Helen Reddy', 'Candle On The Water', 'Tegan', 'Levine')
602
(80, 602, 'Taylor Swift', 'Our Song', 'Tegan', 'Levine')
602
(80, 602, 'Sean Paul', 'Baby Boy [feat. Beyonce]', 'Tegan', 'Levine')
582
(15, 582, 'Soundgarden', 'Black Hole Sun', 'Lily', 'Koch')
602
(80, 602, 'The Killers', 'Human', 'Tegan', 'Levine')
582
(15, 582, 'Amy Winehouse', 'Addicted', 'Lily', 'Koch')
582
(15, 582, 'Steve Anderson', 'Air', 'Lily', 'Koch')
582

In [32]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "drop TABLE IF EXISTS user_by_song "
session.execute(query)
query = "CREATE TABLE IF NOT EXISTS user_by_song "
query = query + """(
firstName text, lastName text, song text, 
PRIMARY KEY(song))
"""
try:
    session.execute(query)
except Exception as e:
    print(e) 
                    

In [33]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_by_song (firstName, lastName, song)"
        query = query + "VALUES (%s, %s, %s)"
        print((line[1], line[4], line[9]))
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[1], line[4], line[9]))


('Ryan', 'Smith', 'Sehr kosmisch')
('Ryan', 'Smith', 'The Big Gundown')
('Ryan', 'Smith', 'Marry Me')
('Samuel', 'Gonzalez', 'Blackbird')
('Tegan', 'Levine', 'Best Of Both Worlds (Remastered Album Version)')
('Tegan', 'Levine', 'Call Me If You Need Me')
('Tegan', 'Levine', 'Home')
('Tegan', 'Levine', 'OMG')
('Tegan', 'Levine', 'Candle On The Water')
('Tegan', 'Levine', 'Our Song')
('Tegan', 'Levine', 'Baby Boy [feat. Beyonce]')
('Lily', 'Koch', 'Black Hole Sun')
('Tegan', 'Levine', 'Human')
('Lily', 'Koch', 'Addicted')
('Lily', 'Koch', 'Air')
('Lily', 'Koch', 'Superbeast')
('Lily', 'Koch', 'I Remember')
('Ryan', 'Smith', "Don't Be Stupid (You Know I Love You)")
('Ryan', 'Smith', 'We throw parties_ you throw knives')
('Chloe', 'Cuevas', 'How Can I Live (Spanish version) (Album Version)')
('Chloe', 'Cuevas', 'Thugs Like Me')
('Chloe', 'Cuevas', 'Killing In The Name')
('Chloe', 'Cuevas', 'Epilogue')
('Chloe', 'Cuevas', 'Endlessly')
('Chloe', 'Cuevas', 'Torches')
('Mohammad', 'Rodriguez', 

In [34]:
query = """
SELECT firstName, lastName
FROM user_by_song
WHERE song = 'All Hands Against His Own'
"""
query = session.execute(query)
print(query)
for row in query:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')


### Drop the tables before closing out the sessions

In [35]:
## TO-DO: Drop the table before closing out the sessions

### Close the session and cluster connection¶

In [36]:
session.shutdown()
cluster.shutdown()